### Python based viewer tool for "Probabilistic Reasoning over time", EDAP01 Artificial Intelligence
This notebook has been provided to you by Alexander Dürr, teaching assistant on the course, spring term 2021. It is based on the ideas and structure of the original Java skeleton for this assignment, provided by Elin A. Topp. Contact us (elin_anna.topp at cs.lth.se) in case you need help!

### Note: Installing and activating widgets for Jupyter Notebook
To be able to display the visualization (dashboard,animations,etc.) you have to initially install the package  if you don't have it yet

### Note: Use Jupyter Lab for programming, Jupyter Notebook for visualization (optional)
This command only enables the extension for jupyter notebook and not in jupyter lab! You can edit from the comfort of jupyter lab though and when you feel like using the widgets just go to

Menu bar > Help > Launch Classic Notebook

## Here we go... inspecting the models, option 1

### Your main job will be in the file Filters.py and in the last cell of this notebook, this is only to understand the models

In _models_, all the actually interesting stuff is located, and in _Filters_ you should write your own code. Note that the visualization (next cell on inspecting the models, option 2) assumes to have access to an object of type _Localizer_ which in turn expects the filtering / smoothing to happen in _Filters.HMM_Filters.filter(sensorR)_. This means that IF you want to make use of the visualisation in grid-view (below!) also for the actual tracking, you MUST implement the filtering in _Filters.HMM_Filter.filter(sensorR)_ (or make changes in _Localizer.Localizer.update()_)



In [ ]:
from models import *
from view_control.Localizer import Localizer

# Testing the models, e.g., for an 4x8 grid

states = StateModel( 4, 8)
loc = Localizer( states, 1)
tMat = loc.get_transition_model()
sVecs = loc.get_observation_model()
tMat.plot_T()
sVecs.plot_o_diags()
print(sVecs.get_o_reading(0))
print(sVecs.get_o_reading(None))

print(loc.update())


## Here we go again... inspecting the models, option 2

### Your implementation job will still be in the file Filters.py, this is only to understand the models AND to get a glimpse of a tracking run (but it is slow)

In _models_, all the actually interesting stuff is located, and in _Filters_ you should write your own code. Note that this visualization assumes to have access to an object of type Localizer which in turn expects the filtering / smoothing to happen in _Filters.HMM_Filters.filter()_. This means that IF you want to make use of the visualisation in grid-view also for the actual tracking, you MUST implement the filtering in Filters.HMM_Filter.filter() (or make respective changes in _Localizer.Localizer.update()_)

### Your Task 1)
#### Inspect the visualisation of the models by running the cell and experimenting with the GUI, in particular compare and explain the different versions of the sensor model (see more detailed instructions for the task and report)

In [1]:
# In view_control.Dashboard, there is simply the handling of all the thread based visualisation provided, 
# no changes needed, but feel free...

from view_control import *

ROWS = 4
COLS = 4

# The dashboard creates a state model of the dimensions given by ROWS and COLS, sets up the respective 
# Transition and Observation models, as well as an instance of class Localizer. The Localizer calls at the 
# moment a stubb method toDo.Filters.HMMFilter.filter(sensorReading), which just sends back the original 
# probability distribution - no filtering is done. It is your task to implement something useful there.

# Non-uniform failure is the default sensor, sensor 0. Change to 1 if that is your default 
# (uniform sensor failure)
sensorType = 0
dash = Dashboard(ROWS, COLS, sensorType)
display(dash.db)


12 12 [ 0  3  6  9 12]
12 12 [ 0  3  6  9 12]
true pose = <2, 3, 0>, sensed nothing
nbr of moves: 1, avg error: 5.0, nbr correct guesses: 0
12 12 [ 0  3  6  9 12]
true pose = <3, 3, 0>, sensed nothing
nbr of moves: 2, avg error: 5.5, nbr correct guesses: 0
12 12 [ 0  3  6  9 12]
true pose = <3, 2, 3>, sensed position = <1, 3>, guessed position = <0, 2>
nbr of moves: 3, avg error: 4.666666666666667, nbr correct guesses: 0
12 12 [ 0  3  6  9 12]
true pose = <3, 1, 3>, sensed nothing
nbr of moves: 4, avg error: 4.5, nbr correct guesses: 0
12 12 [ 0  3  6  9 12]
true pose = <3, 0, 3>, sensed nothing
nbr of moves: 5, avg error: 4.2, nbr correct guesses: 0
12 12 [ 0  3  6  9 12]
true pose = <3, 1, 1>, sensed position = <2, 1>, guessed position = <2, 1>
nbr of moves: 6, avg error: 3.6666666666666665, nbr correct guesses: 0
12 12 [ 0  3  6  9 12]
true pose = <3, 2, 1>, sensed nothing
nbr of moves: 7, avg error: 3.857142857142857, nbr correct guesses: 0
12 12 [ 0  3  6  9 12]
true pose = <3, 1,

12 12 [ 0  3  6  9 12]
true pose = <3, 3, 1>, sensed nothing
nbr of moves: 58, avg error: 2.7241379310344827, nbr correct guesses: 4
12 12 [ 0  3  6  9 12]
true pose = <3, 2, 3>, sensed nothing
nbr of moves: 59, avg error: 2.76271186440678, nbr correct guesses: 4
12 12 [ 0  3  6  9 12]
true pose = <3, 1, 3>, sensed nothing
nbr of moves: 60, avg error: 2.783333333333333, nbr correct guesses: 4
12 12 [ 0  3  6  9 12]
true pose = <3, 0, 3>, sensed nothing
nbr of moves: 61, avg error: 2.7868852459016393, nbr correct guesses: 4
12 12 [ 0  3  6  9 12]
true pose = <2, 0, 2>, sensed position = <0, 1>, guessed position = <0, 1>
nbr of moves: 62, avg error: 2.7903225806451615, nbr correct guesses: 4
12 12 [ 0  3  6  9 12]
true pose = <2, 1, 1>, sensed position = <2, 1>, guessed position = <2, 1>
nbr of moves: 63, avg error: 2.746031746031746, nbr correct guesses: 5
12 12 [ 0  3  6  9 12]
true pose = <2, 0, 3>, sensed nothing
nbr of moves: 64, avg error: 2.734375, nbr correct guesses: 5
12 12 [ 0

12 12 [ 0  3  6  9 12]
true pose = <0, 0, 2>, sensed position = <1, 0>, guessed position = <1, 0>
nbr of moves: 112, avg error: 2.4285714285714284, nbr correct guesses: 9
12 12 [ 0  3  6  9 12]
true pose = <1, 0, 0>, sensed nothing
nbr of moves: 113, avg error: 2.415929203539823, nbr correct guesses: 9
12 12 [ 0  3  6  9 12]
true pose = <0, 0, 2>, sensed position = <0, 0>, guessed position = <0, 0>
nbr of moves: 114, avg error: 2.3947368421052633, nbr correct guesses: 10
12 12 [ 0  3  6  9 12]
true pose = <0, 1, 1>, sensed nothing
nbr of moves: 115, avg error: 2.382608695652174, nbr correct guesses: 10
12 12 [ 0  3  6  9 12]
true pose = <0, 2, 1>, sensed nothing
nbr of moves: 116, avg error: 2.3793103448275863, nbr correct guesses: 10
12 12 [ 0  3  6  9 12]
true pose = <0, 3, 1>, sensed nothing
nbr of moves: 117, avg error: 2.3846153846153846, nbr correct guesses: 10
12 12 [ 0  3  6  9 12]
true pose = <1, 3, 0>, sensed position = <0, 2>, guessed position = <0, 2>
nbr of moves: 118, avg

## Write your own "main" here (without the viewer)

### Your task 2)
#### Implement both Forward Filtering and Fixed-lag FB Smoothing with k = t-5 (a sequence length of 5)
#### Do evaluations (500 steps should be enough also for the bigger grids) according to the following, adapt the number of steps or the size of the grid if needed, but comment on it
#### Provide plots of the Manhattan distance over time

1)
* Forward Filtering with non-uniform sensor failure on 8x8 grid against
* Sensor output only (non-uniform sensor failure, count sensor failures to get the average frequency, but do not count those steps into the avg Manhattan distance) on 8x8 grid 


2)
* Forward Filtering with non-uniform sensor failure on 4x4 grid against
* Forward Filtering with uniform sensor failure on 4x4 grid


3)
* Forward Filtering with non-uniform sensor failure on 16x20 grid against
* Forward Filtering with uniform sensor failure on 16x20 grid


4)
* Forward Filtering with non-uniform sensor failure on 10x10 grid against
* Smoothing (fixed lag forward-backward smoothing) with k = t-5 (five steps for b) and non-uniform sensor failure on 10x10 grid

#### OBS: obviously, each pair-wise evaluation should be run based on the same true trajectory (cases 1, 2, 3) or same trajectory AND same sensor reading sequence (for case 4).

In [ ]:
# In this cell, you can write your own "main" to run and evaluate your 
# implementation without using the visualisation above (should be considerably faster!)

from models import *
from Filters import HMMFilter

import numpy as np
import matplotlib.pyplot as plt
import random

# ...  